<a href="https://colab.research.google.com/github/jposyluzny/ENEL645/blob/main/Lectures/Week%203/MNISTDigitsClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step-by-step MNIST Digits Classification - Fully Connected Neural Networks
1. Load dataset, explore it (display images, mean, min, max values, etc.) and split it into train, validation and test sets
2. Data scaling
3. One hot encoding
4. Define your model, cost function, optimizer, learning rate
5. Define your callbacks (save your model, patience, etc.)
6. Train your model
 <br> 6.1 If you are satisfied with the train and validation performance go to the next step
  <br> 6.2 If you are not satisfied with the train and validation performance go back to step 5

7. Test your model on the test and extract relevant metrics

In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf